# AI解读民国文学：古典与现代的文化融合分析

**基于深度学习的民国作家风格识别与文化成分研究**

---

## 课程简介

本实验将带你体验如何使用人工智能技术分析民国时期文学作品，探索作家风格与中西文化融合的规律。

通过本实验，你将：

1. 了解**文本向量化**技术如何将文学作品转化为数学表示
2. 体验**猜作家游戏**：AI能否准确识别不同作家的风格？
3. 掌握**文化成分分析**：量化文本中的古典与现代元素
4. 探索**风格空间可视化**：在二维平面上看清作家之间的风格距离
5. 使用**AI解说员**：让大语言模型解读风格差异的文化原因
6. 尝试**AI模仿作家**：生成特定作家风格的文字

---

## 数据来源与处理说明

| 数据集 | 来源 | 用途 |
|--------|------|------|
| luxun | GitHub开源项目 | 鲁迅全集（呐喊、彷徨、野草等） |
| literature-books | GitHub开源项目 | 老舍、周作人、季羡林等作家作品 |
| chinese-poetry | GitHub开源项目 | 论语、诗经（古典文言参照） |

**研究维度说明**：
- 古典性：文言词汇占比、古典意象使用频率
- 现代性：白话程度、外来概念词汇占比
- 本实验聚焦于探索民国作家如何在传统与现代之间寻找平衡

---

## 目录

- [第零部分: 环境配置与准备](#part0)
- [第一部分: 数据下载与加载](#part1)
- [第二部分: 数据探索与预处理](#part2)
- [第三部分: AI文本向量化](#part3)
- [第四部分: 猜作家游戏](#part4)
- [第五部分: 文化成分分析仪](#part5)
- [第六部分: 风格空间可视化](#part6)
- [第七部分: 作家风格热力图](#part7)
- [第八部分: AI风格解说员](#part8)
- [第九部分: AI模仿作家](#part9)
- [附录: 常见问题与拓展](#appendix)

---
<a id='part0'></a>
# 第零部分: 环境配置与准备

In [1]:
#@title ## 0.1 检查运行环境 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，检查GPU环境。
#@markdown ---

import sys
import os

print("=" * 60)
print(" 环境检测")
print("=" * 60)
print(f" Python 版本: {sys.version.split()[0]}")

IN_COLAB = 'google.colab' in sys.modules
print(f" 运行环境: {'Google Colab' if IN_COLAB else '本地环境'}")

try:
    import subprocess
    result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
                          capture_output=True, text=True)
    if result.returncode == 0:
        gpu_info = result.stdout.strip()
        print(f" GPU 可用: {gpu_info}")
        if 'A100' in gpu_info:
            print(" [推荐] A100 GPU 检测到，可运行完整功能")
        else:
            print(" [提示] 建议使用更高性能 GPU 以获得最佳体验")
    else:
        print(" 未检测到 GPU")
        print(" 请在菜单中选择: 运行时 -> 更改运行时类型 -> GPU")
except:
    print(" 无法检测 GPU")

print("=" * 60)

 环境检测
 Python 版本: 3.12.12
 运行环境: Google Colab
 GPU 可用: NVIDIA A100-SXM4-80GB, 81920 MiB
 [推荐] A100 GPU 检测到，可运行完整功能


In [2]:
#@title ## 0.2 安装必要的库 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，安装本实验需要的Python库。
#@markdown ---

print(" 正在安装依赖...")
print("-" * 40)

import subprocess
import sys
import warnings
warnings.filterwarnings('ignore')

packages = [
    ('sentence-transformers', 'sentence_transformers'),
    ('umap-learn', 'umap'),
    ('plotly', 'plotly'),
    ('gradio', 'gradio'),
    ('scikit-learn', 'sklearn'),
    ('jieba', 'jieba'),
    ('wordcloud', 'wordcloud'),
    ('seaborn', 'seaborn'),
    ('transformers', 'transformers'),
    ('accelerate', 'accelerate'),
]

for i, (pkg_name, import_name) in enumerate(packages, 1):
    print(f"[{i}/{len(packages)}] 检查 {pkg_name}...")
    try:
        __import__(import_name)
        print(f"    已安装")
    except ImportError:
        print(f"    正在安装...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg_name])
        print(f"    完成")

print("-" * 40)
print(" 所有依赖安装完成!")

 正在安装依赖...
----------------------------------------
[1/10] 检查 sentence-transformers...
    已安装
[2/10] 检查 umap-learn...
    已安装
[3/10] 检查 plotly...
    已安装
[4/10] 检查 gradio...
    已安装
[5/10] 检查 scikit-learn...
    已安装
[6/10] 检查 jieba...
    已安装
[7/10] 检查 wordcloud...
    已安装
[8/10] 检查 seaborn...
    已安装
[9/10] 检查 transformers...
    已安装
[10/10] 检查 accelerate...
    已安装
----------------------------------------
 所有依赖安装完成!


In [4]:
#@title ## 0.3 导入库并配置环境 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，导入所有需要的库。
#@markdown ---

import json
import glob
import os
import re
import numpy as np
import pandas as pd
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

import jieba
from sentence_transformers import SentenceTransformer

import umap
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

import gradio as gr

np.random.seed(42)

print("=" * 60)
print(" 库导入完成")
print("=" * 60)
print(f" NumPy: {np.__version__}")
print(f" Pandas: {pd.__version__}")
print(" 分词工具: jieba")
print(" 向量化: sentence-transformers")
print("=" * 60)

 库导入完成
 NumPy: 2.0.2
 Pandas: 2.2.2
 分词工具: jieba
 向量化: sentence-transformers


---
<a id='part1'></a>
# 第一部分: 数据下载与加载

In [5]:
#@title ## 1.1 下载鲁迅全集 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，从GitHub下载鲁迅全集。
#@markdown
#@markdown 包含：呐喊、彷徨、野草、朝花夕拾、杂文等
#@markdown ---

LUXUN_DIR = 'luxun'

if not os.path.exists(LUXUN_DIR):
    print(" 正在下载鲁迅全集...")
    print("-" * 40)
    !git clone --depth 1 https://github.com/Ac-heron/luxun.git
    print("-" * 40)
    print(" 下载完成!")
else:
    print(" 鲁迅全集已存在，跳过下载")

# 统计文件
md_files = glob.glob(f"{LUXUN_DIR}/**/*.md", recursive=True)
print(f" 找到 {len(md_files)} 个文本文件")

 鲁迅全集已存在，跳过下载
 找到 446 个文本文件


In [6]:
#@title ## 1.2 下载其他民国作家作品 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，下载老舍、周作人、季羡林等作家作品。
#@markdown ---

LIT_DIR = 'literature-books'

if not os.path.exists(LIT_DIR):
    print(" 正在下载文学作品集...")
    print("-" * 40)
    !git clone --depth 1 https://github.com/hankinghu/literature-books.git
    print("-" * 40)
    print(" 下载完成!")
else:
    print(" 文学作品集已存在，跳过下载")

txt_files = glob.glob(f"{LIT_DIR}/**/*.txt", recursive=True)
print(f" 找到 {len(txt_files)} 个文本文件")

 正在下载文学作品集...
----------------------------------------
Cloning into 'literature-books'...
remote: Enumerating objects: 474, done.
remote: Counting objects: 100% (474/474), done.
remote: Compressing objects: 100% (458/458), done.
remote: Total 474 (delta 10), reused 465 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (474/474), 499.27 MiB | 39.89 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Updating files: 100% (479/479), done.
----------------------------------------
 下载完成!
 找到 272 个文本文件


In [7]:
#@title ## 1.3 下载古典文献（论语、诗经） { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，下载古典文献作为文言参照。
#@markdown
#@markdown 用途：计算文本的「古典性」指标
#@markdown ---

POETRY_DIR = 'chinese-poetry'

if not os.path.exists(POETRY_DIR):
    print(" 正在下载古典文献数据集...")
    print("-" * 40)
    !git clone --depth 1 https://github.com/chinese-poetry/chinese-poetry.git
    print("-" * 40)
    print(" 下载完成!")
else:
    print(" 古典文献已存在，跳过下载")

# 检查论语和诗经
lunyu_path = f"{POETRY_DIR}/lunyu"
shijing_path = f"{POETRY_DIR}/shijing"

if os.path.exists(lunyu_path):
    print(f" 论语数据: 已找到")
if os.path.exists(shijing_path):
    print(f" 诗经数据: 已找到")

 正在下载古典文献数据集...
----------------------------------------
Cloning into 'chinese-poetry'...
remote: Enumerating objects: 2277, done.
remote: Counting objects: 100% (2277/2277), done.
remote: Compressing objects: 100% (1166/1166), done.
remote: Total 2277 (delta 1143), reused 1340 (delta 1104), pack-reused 0 (from 0)
Receiving objects: 100% (2277/2277), 94.60 MiB | 13.64 MiB/s, done.
Resolving deltas: 100% (1143/1143), done.
Updating files: 100% (2285/2285), done.
----------------------------------------
 下载完成!


In [8]:
#@title ## 1.4 加载鲁迅作品 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，解析鲁迅全集的文件。
#@markdown ---

def load_luxun_works(base_dir='luxun'):
    """加载鲁迅作品，按作品集和篇目组织"""
    works = []

    # 主要作品集目录映射
    collections = {
        '呐喊': '小说',
        '彷徨': '小说',
        '故事新编': '小说',
        '野草': '散文诗',
        '朝花夕拾': '散文',
        '热风': '杂文',
        '坟': '杂文',
        '华盖集': '杂文',
        '华盖集续编': '杂文',
        '而已集': '杂文',
        '三闲集': '杂文',
        '二心集': '杂文',
        '南腔北调集': '杂文',
        '伪自由书': '杂文',
        '准风月谈': '杂文',
        '花边文学': '杂文',
        '且介亭杂文': '杂文',
    }

    for md_file in glob.glob(f"{base_dir}/**/*.md", recursive=True):
        # 跳过README等非作品文件
        if 'README' in md_file or 'readme' in md_file:
            continue

        try:
            with open(md_file, 'r', encoding='utf-8') as f:
                content = f.read()

            # 提取标题（通常是第一个#标题）
            title_match = re.search(r'^#\s*(.+)$', content, re.MULTILINE)
            title = title_match.group(1).strip() if title_match else os.path.basename(md_file).replace('.md', '')

            # 清理内容：移除Markdown标记
            text = re.sub(r'^#+\s*.*$', '', content, flags=re.MULTILINE)  # 移除标题
            text = re.sub(r'\[([^\]]+)\]\([^)]+\)', r'\1', text)  # 移除链接
            text = re.sub(r'[\*_]{1,2}([^\*_]+)[\*_]{1,2}', r'\1', text)  # 移除粗体斜体
            text = re.sub(r'\n{3,}', '\n\n', text)  # 合并多余空行
            text = text.strip()

            if len(text) < 100:  # 跳过太短的文件
                continue

            # 判断所属作品集
            collection = '其他'
            genre = '杂文'
            for coll_name, coll_genre in collections.items():
                if coll_name in md_file:
                    collection = coll_name
                    genre = coll_genre
                    break

            works.append({
                'author': '鲁迅',
                'title': title,
                'collection': collection,
                'genre': genre,
                'content': text,
                'char_count': len(text),
                'file_path': md_file
            })

        except Exception as e:
            continue

    return pd.DataFrame(works)

print(" 正在加载鲁迅作品...")
print("-" * 40)

df_luxun = load_luxun_works()

print(f" 加载完成: {len(df_luxun)} 篇作品")
print("\n 各作品集统计:")
for coll, count in df_luxun['collection'].value_counts().head(10).items():
    print(f"   {coll}: {count} 篇")
print("-" * 40)

 正在加载鲁迅作品...
----------------------------------------
 加载完成: 445 篇作品

 各作品集统计:
   其他: 134 篇
   华盖集: 68 篇
   热风: 42 篇
   二心集: 38 篇
   三闲集: 34 篇
   而已集: 31 篇
   野草: 27 篇
   坟: 20 篇
   呐喊: 16 篇
   朝花夕拾: 13 篇
----------------------------------------


In [9]:
#@title ## 1.5 加载其他作家作品 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，加载其他民国作家作品并过滤无关内容。
#@markdown ---

#清理文本中的乱码
def clean_text(text):
    """清理网络文本中的广告水印"""
    if not text:
        return ""
    patterns = [
        r'[！!]?小[@＠]说[#＃]?txt[＄$]?天[＾^]?堂[＆&]?',
        r'www\.[a-zA-Z0-9]+\.(com|net|org|cn)',
        r'http[s]?://[^\s]+',
        r'【[^】]*下载[^】]*】',
        r'本书来自[^\n]+',
        r'更多.*请访问[^\n]+',
        r'手打吧|无弹窗|全文阅读',
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    text = re.sub(r'\n{3,}', '\n\n', text)
    return text.strip()

def identify_author(filename):
    """根据文件名识别作者"""
    for author, keywords in AUTHOR_PATTERNS.items():
        for kw in keywords:
            if kw in filename:
                return author
    return None

def should_exclude(filename):
    """判断是否应排除"""
    for kw in EXCLUDE_KEYWORDS:
        if kw in filename:
            return True
    return False

def load_literature_books(base_dir='literature-books'):
    """加载文学作品集中的作品"""
    works = []

    #筛选数据集里有用的文章
    # 作者识别映射
    author_patterns = {
        '老舍': ['老舍', '四世同堂', '骆驼祥子', '茶馆'],
        '季羡林': ['季羡林', '牛棚杂忆', '留德十年'],
        '三毛': ['三毛', '撒哈拉', '我的宝贝', '梦里花落知多少', '哭泣的骆驼', '温柔的夜', '雨季不再来', '稻草人手记', '闹学记', '送你一匹马', '万水千山走遍'],
        '朱自清': ['朱自清', '背影', '荷塘月色'],
        '郁达夫': ['郁达夫', '沉沦', '春风沉醉的晚上'],
        '巴金': ['巴金', '寒夜', '随想录'],
        '萧红': ['萧红', '呼兰河传'],
        '许地山': ['许地山', '缀网劳蛛', '落花生'],
        '冰心': ['冰心', '繁星', '春水', '关于女人', '山中杂记'],
        '林徽因': ['林徽因'],
        '胡适': ['胡适'],
        '史铁生': ['史铁生', '命若琴弦', '病隙碎笔', '我的遥远的清平湾', '插队的故事', '记忆与印象'],
        '杨绛': ['杨绛', '走到人生边上'],
        '冯骥才': ['冯骥才', '俗世奇人'],
        '章诒和': ['章诒和', '往事并不如烟'],
    }

    # 排除关键词（非民国文学）
    exclude_keywords = [
        '金庸', '古龙', '梁羽生', '天龙八部', '笑傲江湖', '陆小凤',  # 武侠
        '证券', '经济', '商道', '理论', '管理', '投资', '定律', '货币战争', '营销', '财富', '穷爸爸', '富爸爸',  # 商业
        '萨缪尔森', '毛姆', '苏菲', '忏悔录', '人生的枷锁', '百万英镑', '钢铁是怎样', '凡高传', '卓别林',  # 外国
        '时间简史', '记忆力', '学习技巧', '博赞', '人类简史', '全球通史',  # 科普
        '红楼梦', '三国', '水浒', '西游', '曾国藩', '李自成', '张居正', '雍正', '明朝那些事',  # 古典/历史
        '鲁迅全集', '务虚笔记',  # 重复或质量差
        '莫言', '檀香刑', '蛙', '丰乳肥臀', '生死疲劳',  # 当代作家
        '盗墓笔记', '狼图腾', '亮剑', '白鹿原', '血色浪漫',  # 当代小说
        '戴旭', '奥巴马', '马云',  # 非文学
        '周作人', #历史问题存在争议
    ]

    for txt_file in glob.glob(f"{base_dir}/**/*.txt", recursive=True):
        try:
            filename = os.path.basename(txt_file)
            filepath = txt_file

            # 检查是否应该排除
            should_exclude = False
            for keyword in exclude_keywords:
                if keyword in filename or keyword in filepath:
                    should_exclude = True
                    break
            if should_exclude:
                continue

            # 尝试多种编码
            content = None
            for encoding in ['utf-8', 'gbk', 'gb2312', 'gb18030']:
                try:
                    with open(txt_file, 'r', encoding=encoding) as f:
                        content = f.read()
                    break
                except:
                    continue

            if content is None:
                continue

            # 清理广告水印
            content = clean_text(content)

            if len(content) < 500:
                continue

            # 识别作者
            author = '未知'
            for auth, patterns in author_patterns.items():
                for pattern in patterns:
                    if pattern in filename or pattern in filepath:
                        author = auth
                        break
                if author != '未知':
                    break

            # 只保留能识别作者的作品
            if author == '未知':
                continue

            title = filename.replace('.txt', '')

            works.append({
                'author': author,
                'title': title,
                'collection': title,
                'genre': '散文/小说',
                'content': content,
                'char_count': len(content),
                'file_path': txt_file
            })

        except Exception as e:
            continue

    return pd.DataFrame(works)

print(" 正在加载其他作家作品...")
print("-" * 40)

df_others = load_literature_books()

print(f" 加载完成: {len(df_others)} 篇作品")
print("\n 各作家统计:")
for author, count in df_others['author'].value_counts().items():
    print(f"   {author}: {count} 篇")
print("-" * 40)

 正在加载其他作家作品...
----------------------------------------
 加载完成: 37 篇作品

 各作家统计:
   三毛: 10 篇
   史铁生: 5 篇
   郁达夫: 4 篇
   季羡林: 3 篇
   冯骥才: 2 篇
   许地山: 2 篇
   冰心: 2 篇
   朱自清: 2 篇
   杨绛: 1 篇
   林徽因: 1 篇
   胡适: 1 篇
   老舍: 1 篇
   巴金: 1 篇
   萧红: 1 篇
   章诒和: 1 篇
----------------------------------------


In [10]:
#@title ## 1.6 加载古典文献 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，加载古典文献作为参照。
#@markdown
#@markdown 包括：论语、诗经
#@markdown ---

import urllib.request

def load_classical_from_cdn():
    """从CDN下载古典文献"""
    classical_texts = []

    # CDN基础地址
    base_url = "https://unpkg.com/chinese-poetry/chinese-poetry"

    # 1. 论语
    print(" 正在下载论语...")
    try:
        url = f"{base_url}/lunyu/lunyu.json"
        with urllib.request.urlopen(url, timeout=30) as response:
            data = json.loads(response.read().decode('utf-8'))
        for item in data:
            if 'paragraphs' in item:
                content = ''.join(item['paragraphs'])
                classical_texts.append({
                    'author': '论语',
                    'title': item.get('chapter', '论语'),
                    'collection': '论语',
                    'genre': '经典',
                    'content': content,
                    'char_count': len(content)
                })
        print(f"   论语: {len([t for t in classical_texts if t['collection']=='论语'])} 篇")
    except Exception as e:
        print(f"   论语下载失败: {e}")

    # 2. 诗经
    print(" 正在下载诗经...")
    try:
        url = f"{base_url}/shijing/shijing.json"
        with urllib.request.urlopen(url, timeout=30) as response:
            data = json.loads(response.read().decode('utf-8'))
        count = 0
        for item in data:
            if 'content' in item:
                content = ''.join(item['content']) if isinstance(item['content'], list) else item['content']
                classical_texts.append({
                    'author': '诗经',
                    'title': item.get('title', '诗经'),
                    'collection': '诗经',
                    'genre': '诗歌',
                    'content': content,
                    'char_count': len(content)
                })
                count += 1
        print(f"   诗经: {count} 篇")
    except Exception as e:
        print(f"   诗经下载失败: {e}")

    return pd.DataFrame(classical_texts)

print(" 正在加载古典文献...")
print("-" * 40)

df_classical = load_classical_from_cdn()

print("-" * 40)
print(f" 加载完成: {len(df_classical)} 篇")
if len(df_classical) > 0:
    print("\n 各文献统计:")
    for coll, count in df_classical['collection'].value_counts().items():
        print(f"   {coll}: {count} 篇")
else:
    print(" [警告] 未能加载古典文献")
print("-" * 40)

 正在加载古典文献...
----------------------------------------
 正在下载论语...
   论语: 20 篇
 正在下载诗经...
   诗经: 305 篇
----------------------------------------
 加载完成: 325 篇

 各文献统计:
   诗经: 305 篇
   论语: 20 篇
----------------------------------------


---
<a id='part2'></a>
# 第二部分: 数据探索与预处理

In [11]:
#@title ## 2.1 合并所有数据 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，合并所有作家的作品数据。
#@markdown ---

# 合并民国作家数据
df_modern = pd.concat([df_luxun, df_others], ignore_index=True)
df_modern['era'] = '民国'

# 标记古典文献
df_classical['era'] = '古典'

# 合并所有数据
df_all = pd.concat([df_modern, df_classical], ignore_index=True)

print("=" * 60)
print(" 数据合并完成")
print("=" * 60)
print(f" 总作品数: {len(df_all)}")
print(f" 民国作品: {len(df_modern)}")
print(f" 古典文献: {len(df_classical)}")
print("\n 民国作家分布:")
for author, count in df_modern['author'].value_counts().items():
    print(f"   {author}: {count} 篇")
print("=" * 60)

 数据合并完成
 总作品数: 807
 民国作品: 482
 古典文献: 325

 民国作家分布:
   鲁迅: 445 篇
   三毛: 10 篇
   史铁生: 5 篇
   郁达夫: 4 篇
   季羡林: 3 篇
   冯骥才: 2 篇
   许地山: 2 篇
   冰心: 2 篇
   朱自清: 2 篇
   杨绛: 1 篇
   林徽因: 1 篇
   胡适: 1 篇
   老舍: 1 篇
   巴金: 1 篇
   萧红: 1 篇
   章诒和: 1 篇


In [12]:
#@title ## 2.2 文本预处理与分段 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，将长文本分割成适合AI处理的段落。
#@markdown
#@markdown 处理策略：
#@markdown - 每段约500-1000字
#@markdown - 保持段落语义完整性
#@markdown ---

def split_text_to_segments(text, min_len=200, max_len=800):
    """将长文本分割成段落"""
    # 按自然段落分割
    paragraphs = re.split(r'\n\s*\n', text)

    segments = []
    current_segment = ""

    for para in paragraphs:
        para = para.strip()
        if not para:
            continue

        if len(current_segment) + len(para) < max_len:
            current_segment += para + "\n"
        else:
            if len(current_segment) >= min_len:
                segments.append(current_segment.strip())
            current_segment = para + "\n"

    # 添加最后一段
    if len(current_segment) >= min_len:
        segments.append(current_segment.strip())

    return segments

print(" 正在分割文本...")
print("-" * 40)

all_segments = []

for idx, row in df_all.iterrows():
    segments = split_text_to_segments(row['content'])
    for i, seg in enumerate(segments):
        all_segments.append({
            'author': row['author'],
            'title': row['title'],
            'collection': row['collection'],
            'genre': row['genre'],
            'era': row['era'],
            'segment_id': i,
            'content': seg,
            'char_count': len(seg)
        })

df_segments = pd.DataFrame(all_segments)

print(f" 分割完成: {len(df_segments)} 个文本段落")
print(f" 平均段落长度: {df_segments['char_count'].mean():.0f} 字")
print("\n 各作家段落数:")
for author, count in df_segments['author'].value_counts().head(10).items():
    print(f"   {author}: {count} 段")
print("-" * 40)

 正在分割文本...
----------------------------------------
 分割完成: 6230 个文本段落
 平均段落长度: 1153 字

 各作家段落数:
   鲁迅: 1807 段
   三毛: 1158 段
   老舍: 1016 段
   史铁生: 370 段
   章诒和: 363 段
   季羡林: 331 段
   朱自清: 208 段
   郁达夫: 189 段
   巴金: 160 段
   许地山: 134 段
----------------------------------------


In [13]:
#@title ## 2.3 采样平衡数据集 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，为每位作家采样相同数量的段落。
#@markdown
#@markdown 目的：避免数据不平衡影响分析结果
#@markdown ---

#@markdown 每位作家采样段落数:
SAMPLES_PER_AUTHOR = 50  #@param {type:"integer"}

print(" 正在采样平衡数据...")
print("-" * 40)

# 只保留有足够样本的作家
author_counts = df_segments['author'].value_counts()
valid_authors = author_counts[author_counts >= 20].index.tolist()

df_sampled = df_segments[df_segments['author'].isin(valid_authors)].groupby('author').apply(
    lambda x: x.sample(n=min(SAMPLES_PER_AUTHOR, len(x)), random_state=42)
).reset_index(drop=True)

print(f" 采样完成: {len(df_sampled)} 个段落")
print(f" 有效作家: {len(valid_authors)} 位")
print("\n 采样后各作家段落数:")
for author, count in df_sampled['author'].value_counts().items():
    print(f"   {author}: {count} 段")
print("-" * 40)

 正在采样平衡数据...
----------------------------------------
 采样完成: 808 个段落
 有效作家: 17 位

 采样后各作家段落数:
   三毛: 50 段
   冯骥才: 50 段
   冰心: 50 段
   史铁生: 50 段
   季羡林: 50 段
   巴金: 50 段
   朱自清: 50 段
   杨绛: 50 段
   林徽因: 50 段
   章诒和: 50 段
   老舍: 50 段
   许地山: 50 段
   鲁迅: 50 段
   郁达夫: 50 段
   诗经: 47 段
   萧红: 41 段
   论语: 20 段
----------------------------------------


In [14]:
#@title ## 2.4 数据预览 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，预览各作家的文本样例。
#@markdown ---

print("=" * 60)
print(" 各作家文本样例")
print("=" * 60)

for author in df_sampled['author'].unique()[:5]:
    sample = df_sampled[df_sampled['author'] == author].iloc[0]
    content_preview = sample['content'][:100] + '...' if len(sample['content']) > 100 else sample['content']

    print(f"\n【{author}】《{sample['title']}》")
    print(f"  {content_preview}")
    print("-" * 40)

 各作家文本样例

【三毛】《送你一匹马》
  后来“有问题”的同学很多，刚开始或许是因为说话想为自己的观念站定脚步，所以口气很强硬，胆子又大，真怕无意中的语言可能伤了老师，但她心胸很宽，认真听学生的看法，一点也不计较，只有鼓励，让我们安心的再谈下...
----------------------------------------

【冯骥才】《俗世奇人》
  背头杨真弄不明白，维新怎么会招来这么多麻烦，不过留一个背头，连厕所也进不得。而且是进厕所不行，不进厕所也不行。不知是她把事情扰乱，还是事情把她扰乱。一赌气，她在屋里呆了两个月。慢慢头发长了，恢复了女相...
----------------------------------------

【冰心】《山中杂记》
  我从小爱花，因为院里、屋里、案头经常有花，但是我从来没有侍弄过花！对于花的美的享受，我从来就是一个“不劳而获”者。
　　我的父亲，业余只喜欢种花，无论住到哪里，庭院里一定要开辟一个花畦。我刚懂事时，记...
----------------------------------------

【史铁生】《插队的故事》
  这时候大喇叭里开始“请到太原去的旅客上车”了。那回我们走山西，先要经过太原。车票都是家里逼着买的，我们本打算退几张，每人一张车票实在花钱太多，结果让刘溪的事给搅得上了火车才想起来。
“你什么时候知道的...
----------------------------------------

【季羡林】《牛棚杂忆》
  此外，我还有一个十分不切实际的期待。上面的期待是对在浩劫中遭受痛苦折磨的人们而说的。折磨人甚至把人折磨至死的当时的“造反派”实际上是打砸抢分子的人，为什么不能够把自己折磨人的心理状态和折磨过程也站出来...
----------------------------------------


---
<a id='part3'></a>
# 第三部分: AI文本向量化

使用BGE中文模型将文本转换为向量表示，捕捉语义特征。

In [15]:
#@title ## 3.1 加载中文向量化模型 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，加载BGE中文大模型。
#@markdown
#@markdown 模型: BAAI/bge-large-zh-v1.5
#@markdown - 参数量: 326M
#@markdown - 向量维度: 1024
#@markdown - 中文语义理解能力强
#@markdown ---

print(" 正在加载中文向量化模型...")
print("-" * 40)
print(" 模型: BAAI/bge-large-zh-v1.5")
print(" 首次加载需下载约1.2GB，请耐心等待...")
print("-" * 40)

model = SentenceTransformer('BAAI/bge-large-zh-v1.5')

# 测试模型
test_text = "这是一个测试句子。"
test_embedding = model.encode(test_text)

print("\n" + "=" * 60)
print(" 模型加载完成")
print("=" * 60)
print(f" 向量维度: {len(test_embedding)}")
print(f" 设备: {model.device}")
print("=" * 60)

 正在加载中文向量化模型...
----------------------------------------
 模型: BAAI/bge-large-zh-v1.5
 首次加载需下载约1.2GB，请耐心等待...
----------------------------------------


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]


 模型加载完成
 向量维度: 1024
 设备: cuda:0


In [16]:
#@title ## 3.2 生成文本向量 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，为所有文本段落生成向量表示。
#@markdown ---

print(" 正在生成文本向量...")
print("-" * 40)

texts = df_sampled['content'].tolist()
print(f" 待处理文本: {len(texts)} 段")

# 批量编码
embeddings = model.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True
)

print("-" * 40)
print("=" * 60)
print(" 向量化完成")
print("=" * 60)
print(f" 向量矩阵形状: {embeddings.shape}")
print(f" 每个向量维度: {embeddings.shape[1]}")
print("=" * 60)

 正在生成文本向量...
----------------------------------------
 待处理文本: 808 段


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

----------------------------------------
 向量化完成
 向量矩阵形状: (808, 1024)
 每个向量维度: 1024


In [17]:
#@title ## 3.3 计算作家平均向量 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，计算每位作家的平均风格向量。
#@markdown ---

print(" 正在计算作家平均向量...")
print("-" * 40)

# 将向量添加到数据框
df_sampled['embedding'] = list(embeddings)

# 计算每位作家的平均向量
author_embeddings = {}
for author in df_sampled['author'].unique():
    author_data = df_sampled[df_sampled['author'] == author]
    author_vecs = np.array(author_data['embedding'].tolist())
    author_embeddings[author] = author_vecs.mean(axis=0)

# 转换为矩阵
author_names = list(author_embeddings.keys())
author_embedding_matrix = np.array([author_embeddings[name] for name in author_names])

print(f" 计算完成: {len(author_names)} 位作家")
print(f" 作家向量矩阵: {author_embedding_matrix.shape}")
print("-" * 40)

 正在计算作家平均向量...
----------------------------------------
 计算完成: 17 位作家
 作家向量矩阵: (17, 1024)
----------------------------------------


In [18]:
#@title ## 3.4 计算作家风格相似度 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，计算作家之间的风格相似度矩阵。

#@markdown
#@markdown ---

print(" 正在计算风格相似度...")
print("-" * 40)

# 计算余弦相似度矩阵
similarity_matrix = cosine_similarity(author_embedding_matrix)

# 转换为DataFrame
df_similarity = pd.DataFrame(
    similarity_matrix,
    index=author_names,
    columns=author_names
)

print(" 相似度矩阵计算完成")
print("\n 与鲁迅风格最相似的作家:")
if '鲁迅' in df_similarity.index:
    luxun_sim = df_similarity.loc['鲁迅'].sort_values(ascending=False)
    for name, sim in luxun_sim[1:6].items():
        print(f"   {name}: {sim:.3f}")
print("-" * 40)

 正在计算风格相似度...
----------------------------------------
 相似度矩阵计算完成

 与鲁迅风格最相似的作家:
   季羡林: 0.956
   巴金: 0.950
   朱自清: 0.945
   史铁生: 0.944
   冰心: 0.940
----------------------------------------


---
<a id='part4'></a>
# 第四部分: 猜作家游戏

测试AI能否准确识别不同作家的写作风格。

In [19]:
#@title ## 4.1 猜作家游戏 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，启动「猜作家」互动游戏。
#@markdown
#@markdown ### 游戏规则
#@markdown 1. 系统随机展示一段文本
#@markdown 2. 你先猜测是哪位作家写的
#@markdown 3. 点击"AI预测"查看AI的判断
#@markdown 4. 看看你和AI谁更准确
#@markdown
#@markdown ### 观察要点
#@markdown - AI判断的置信度高低意味着什么？
#@markdown - 哪些作家的风格更容易被识别？
#@markdown - AI的判断依据与你的直觉有何不同？
#@markdown ---

# 准备游戏数据
game_samples = df_sampled.sample(n=min(100, len(df_sampled)), random_state=42).reset_index(drop=True)
current_idx = [0]

def get_new_sample():
    """获取新的随机样本"""
    idx = np.random.randint(0, len(game_samples))
    current_idx[0] = idx
    sample = game_samples.iloc[idx]
    content = sample['content']
    if len(content) > 300:
        content = content[:300] + '...'
    return content

def predict_author():
    """AI预测作家"""
    sample = game_samples.iloc[current_idx[0]]
    true_author = sample['author']

    # 计算与所有作家的相似度
    text_embedding = model.encode(sample['content'], convert_to_numpy=True)
    similarities = {}
    for author, author_emb in author_embeddings.items():
        sim = cosine_similarity([text_embedding], [author_emb])[0][0]
        similarities[author] = sim

    # 排序
    sorted_sims = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    predicted_author = sorted_sims[0][0]
    confidence = sorted_sims[0][1]

    # 构建结果
    result = []
    result.append("=" * 50)
    result.append(" AI预测结果")
    result.append("=" * 50)
    result.append(f"")
    result.append(f" 预测作家: {predicted_author}")
    result.append(f" 置信度: {confidence:.1%}")
    result.append(f"")
    result.append(f" 实际作家: {true_author}")
    result.append(f" 出处: 《{sample['title']}》")
    result.append(f"")

    if predicted_author == true_author:
        result.append(" [正确] AI猜对了!")
    else:
        result.append(f" [错误] AI猜错了")

    result.append("")
    result.append("-" * 50)
    result.append(" Top 5 候选作家:")
    result.append("-" * 50)
    for i, (author, sim) in enumerate(sorted_sims[:5], 1):
        marker = " <-- 正确答案" if author == true_author else ""
        result.append(f" {i}. {author}: {sim:.1%}{marker}")

    return "\n".join(result)

# 创建Gradio界面
with gr.Blocks(title="猜作家游戏") as demo_game:
    gr.Markdown("# 猜作家游戏")
    gr.Markdown("阅读下面的文字，猜猜是哪位作家写的，然后让AI来预测!")

    with gr.Row():
        with gr.Column():
            text_display = gr.Textbox(
                label="文本内容",
                lines=10,
                value=get_new_sample(),
                interactive=False
            )
            with gr.Row():
                new_btn = gr.Button("换一段文字", variant="secondary")
                predict_btn = gr.Button("AI预测", variant="primary")

        with gr.Column():
            result_display = gr.Textbox(
                label="预测结果",
                lines=18,
                interactive=False
            )

    new_btn.click(fn=get_new_sample, outputs=text_display)
    predict_btn.click(fn=predict_author, outputs=result_display)

demo_game.launch(share=True, debug=False, quiet=True)

* Running on public URL: https://436f67a15a81cecccd.gradio.live


---
<a id='part5'></a>
# 第五部分: 文化成分分析仪

量化文本中的古典与现代文化元素。

In [20]:
#@title ## 5.1 文化成分分析仪 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，启动文化成分分析工具。
#@markdown
#@markdown ### 分析维度
#@markdown - **古典词汇占比**: 文言虚词、古典意象的使用频率
#@markdown - **现代概念词占比**: 外来词、新概念词的使用频率
#@markdown - **平均句长**: 句子的平均字数
#@markdown - **情感倾向**: 正面/负面情感词的比例
#@markdown
#@markdown ### 使用建议
#@markdown - 输入一段文字，查看其「文化成分表」
#@markdown - 比较不同作家的文化成分差异
#@markdown - 尝试输入你自己写的文字，看看像哪位作家
#@markdown ### 注意事项
#@markdown
#@markdown - 相似度40%-50%属于正常范围，说明文本有独特性
#@markdown - 同一作家不同体裁的作品风格可能差异很大
#@markdown - 例如鲁迅的杂文与小说风格截然不同，可能被识别为不同作家
#@markdown - 这正是AI分析的有趣之处：它捕捉的是文本特征，而非作者标签
#@markdown ---

# 定义词汇表
CLASSICAL_WORDS = set([
    # 文言虚词
    '之', '乎', '者', '也', '矣', '焉', '哉', '夫', '其', '若', '然', '故',
    '则', '虽', '且', '乃', '于', '与', '以', '为', '而', '或', '亦', '盖',
    # 古典意象
    '明月', '清风', '山水', '烟雨', '流云', '飞鸟', '落花', '残阳', '孤舟',
    '寒江', '古道', '长亭', '短亭', '斜阳', '暮色', '晨曦', '黄昏', '月色',
    # 古典情感词
    '惆怅', '怅然', '萧索', '凄凉', '苍茫', '寂寥', '悠然', '怡然', '悲戚',
])

MODERN_WORDS = set([
    # 外来概念词
    '科学', '民主', '自由', '革命', '进化', '文明', '社会', '经济', '政治',
    '哲学', '艺术', '文学', '历史', '心理', '精神', '思想', '理想', '现实',
    # 现代生活词
    '电灯', '电车', '汽车', '火车', '轮船', '飞机', '电话', '电报', '报纸',
    '杂志', '学校', '医院', '工厂', '银行', '公司', '机关', '政府', '国家',
    # 现代情感词
    '焦虑', '压抑', '绝望', '希望', '奋斗', '抗争', '觉醒', '解放', '启蒙',
])

NEGATIVE_WORDS = set([
    '悲', '苦', '痛', '恨', '怒', '哀', '愁', '忧', '伤', '死', '亡', '绝',
    '黑暗', '绝望', '孤独', '寂寞', '凄凉', '悲哀', '痛苦', '可怜', '可悲',
])

POSITIVE_WORDS = set([
    '喜', '乐', '欢', '爱', '美', '善', '光', '明', '希望', '快乐', '幸福',
    '温暖', '阳光', '春天', '花开', '笑', '甜', '香', '新', '生', '活',
])

def analyze_cultural_components(text):
    """分析文本的文化成分"""
    if not text or len(text.strip()) < 10:
        return "请输入至少10个字的文本"

    # 分词
    words = jieba.lcut(text)
    total_words = len(words)

    if total_words == 0:
        return "无法分析空文本"

    # 统计各类词汇
    classical_count = sum(1 for w in words if w in CLASSICAL_WORDS)
    modern_count = sum(1 for w in words if w in MODERN_WORDS)
    negative_count = sum(1 for w in words if w in NEGATIVE_WORDS)
    positive_count = sum(1 for w in words if w in POSITIVE_WORDS)

    # 计算比例
    classical_ratio = classical_count / total_words
    modern_ratio = modern_count / total_words

    # 计算句子统计
    sentences = re.split(r'[。！？；]', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    avg_sentence_len = len(text) / max(len(sentences), 1)

    # 情感倾向
    if positive_count + negative_count > 0:
        sentiment = (positive_count - negative_count) / (positive_count + negative_count)
    else:
        sentiment = 0

    # 计算与各作家的相似度
    text_embedding = model.encode(text, convert_to_numpy=True)
    similarities = []
    for author, author_emb in author_embeddings.items():
        sim = cosine_similarity([text_embedding], [author_emb])[0][0]
        similarities.append((author, sim))
    similarities.sort(key=lambda x: x[1], reverse=True)

    # 构建进度条
    def make_bar(ratio, width=20):
        filled = int(ratio * width)
        return '[' + '=' * filled + ' ' * (width - filled) + ']'

    # 构建结果
    result = []
    result.append("=" * 50)
    result.append(" 文化成分分析报告")
    result.append("=" * 50)
    result.append(f"")
    result.append(f" 文本长度: {len(text)} 字 | 词数: {total_words}")
    result.append(f" 句子数: {len(sentences)} | 平均句长: {avg_sentence_len:.1f} 字")
    result.append(f"")
    result.append("-" * 50)
    result.append(" 文化成分占比:")
    result.append("-" * 50)
    result.append(f" 古典词汇  {make_bar(classical_ratio)} {classical_ratio:.1%}")
    result.append(f" 现代概念  {make_bar(modern_ratio)} {modern_ratio:.1%}")
    result.append(f"")

    # 情感分析
    if sentiment > 0.2:
        sentiment_desc = "偏积极"
    elif sentiment < -0.2:
        sentiment_desc = "偏消极"
    else:
        sentiment_desc = "中性"
    result.append(f" 情感倾向: {sentiment_desc} ({sentiment:+.2f})")
    result.append(f"")
    result.append("-" * 50)
    result.append(" 风格最相似的作家:")
    result.append("-" * 50)
    for i, (author, sim) in enumerate(similarities[:5], 1):
        result.append(f" {i}. {author}: {sim:.1%}")

    result.append(f"")
    result.append("=" * 50)

    return "\n".join(result)

# 创建Gradio界面
demo_analyzer = gr.Interface(
    fn=analyze_cultural_components,
    inputs=gr.Textbox(
        label="输入文本",
        lines=8,
        placeholder="在此输入要分析的文本..."
    ),
    outputs=gr.Textbox(label="分析结果", lines=25),
    title="文化成分分析仪",
    description="输入任意文本，分析其古典与现代文化成分，并找出风格最相似的作家。",
    examples=[
        ["我翻开历史一查，这历史没有年代，歪歪斜斜的每页上都写着'仁义道德'几个字。我横竖睡不着，仔细看了半夜，才从字缝里看出字来，满本都写着两个字是'吃人'！"],
        ["月色便朦胧在这水气里。淡黑的起伏的连山，仿佛是踊跃的铁的兽脊似的，都远远地向船尾跑去了。"],
        ["人生最苦痛的是梦醒了无路可以走。做梦的人是幸福的；倘没有看出可走的路，最要紧的是不要去惊醒他。"],
    ],
    allow_flagging='never'
)

demo_analyzer.launch(share=True, debug=False, quiet=True)

* Running on public URL: https://984cd77a1adc35da4b.gradio.live


### 思考：为什么AI会"认错"作者？

在使用文化成分分析仪时，你可能会发现一个有趣的现象：输入某位作家的文章，AI却认为最相似的是另一位作家。

例如：
- 鲁迅《狂人日记》的片段 → AI认为最像杨绛
- 鲁迅《社戏》的写景段落 → AI认为最像郁达夫

**这是AI出错了吗？**

不完全是。这恰恰揭示了几个重要的问题：

**1. 作家风格并非铁板一块**

同一位作家在不同时期、不同体裁的作品中，风格可能截然不同：
- 鲁迅的杂文：犀利、讽刺、战斗性强
- 鲁迅的小说：细腻、抒情、富有画面感
- 鲁迅的散文诗：象征、隐晦、充满张力

当AI计算"鲁迅的平均风格"时，这些不同风格被混在一起，反而模糊了特征。

**2. AI捕捉的是文本特征，而非作者标签**

AI不知道"这是鲁迅写的"，它只看到文字本身的特征：
- 句子长度、词汇选择、表达方式、情感色调

两个不同作家写出风格相似的文字，在AI眼中就是"相似"的。

**3. 训练样本的影响**

本实验中：
- 鲁迅有400+篇作品（杂文居多）
- 其他作家只有几篇到十几篇

数据不平衡会影响AI的判断基准。

**思考题**

1. 如果只用鲁迅的小说训练AI，识别《社戏》的准确率会提高吗？为什么？
2. "风格相似"是否意味着存在模仿或影响关系？还有哪些可能的解释？
3. 这种"认错"现象对文学研究有什么启发？

---
<a id='part6'></a>
# 第六部分: 风格空间可视化

将作家投射到二维平面，直观展示风格距离。

In [21]:
#@title ## 6.1 风格空间可视化 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，生成作家风格空间图。
#@markdown
#@markdown ### 解读指南
#@markdown - 点之间的距离代表风格相似度
#@markdown - 距离越近，风格越相似
#@markdown - 颜色区分不同时代（民国 vs 古典）
#@markdown
#@markdown ### 观察要点
#@markdown - 哪些作家"抱团"站在一起？他们有什么共同点？
#@markdown - 鲁迅站在哪里？离谁最近、离谁最远？
#@markdown - 古典文献（红点）和民国作家（蓝点）隔了多远？
#@markdown ---

print(" 正在生成风格空间可视化...")
print("-" * 40)

# 使用UMAP降维
reducer = umap.UMAP(
    n_neighbors=5,
    min_dist=0.3,
    n_components=2,
    random_state=42
)

author_2d = reducer.fit_transform(author_embedding_matrix)

# 创建可视化数据
viz_data = pd.DataFrame({
    'x': author_2d[:, 0],
    'y': author_2d[:, 1],
    'author': author_names,
    'era': [df_sampled[df_sampled['author'] == a]['era'].iloc[0] if a in df_sampled['author'].values else '未知' for a in author_names]
})

# 使用Plotly创建交互式图表
fig = px.scatter(
    viz_data,
    x='x',
    y='y',
    text='author',
    color='era',
    title='作家风格空间可视化',
    labels={'x': '维度1', 'y': '维度2', 'era': '时代'},
    color_discrete_map={'民国': '#1f77b4', '古典': '#d62728'}
)

fig.update_traces(
    textposition='top center',
    marker=dict(size=12)
)

fig.update_layout(
    width=900,
    height=700,
    font=dict(size=14)
)

fig.show()

print("-" * 40)
print(" 可视化完成")
print(" 提示: 可以拖动、缩放、悬停查看详情")

 正在生成风格空间可视化...
----------------------------------------


----------------------------------------
 可视化完成
 提示: 可以拖动、缩放、悬停查看详情


In [22]:
#@title ## 6.2 文本段落风格空间 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，展示所有文本段落在风格空间中的分布。
#@markdown
#@markdown ### 解读指南
#@markdown - 每个小点代表一段文字（约500-1000字）
#@markdown - 同一颜色的点属于同一作家
#@markdown - 上一张图是作家平均值，这张图是每个段落的具体位置
#@markdown
#@markdown ### 观察要点
#@markdown - 哪位作家的点最集中？说明风格稳定统一
#@markdown - 哪位作家的点最分散？说明风格多变
#@markdown - 不同作家的点有没有混在一起？说明风格相近
#@markdown - 鲁迅的点分布范围大吗？这和他体裁多样有关吗？
#@markdown ---

print(" 正在生成文本段落风格空间...")
print("-" * 40)

# 对所有文本段落降维
text_2d = reducer.fit_transform(embeddings)

# 创建可视化数据
text_viz_data = pd.DataFrame({
    'x': text_2d[:, 0],
    'y': text_2d[:, 1],
    'author': df_sampled['author'].values,
    'title': df_sampled['title'].values,
    'era': df_sampled['era'].values
})

# 使用Plotly创建交互式图表
fig2 = px.scatter(
    text_viz_data,
    x='x',
    y='y',
    color='author',
    hover_data=['title'],
    title='文本段落风格空间分布',
    labels={'x': '维度1', 'y': '维度2', 'author': '作家'}
)

fig2.update_traces(marker=dict(size=6, opacity=0.7))

fig2.update_layout(
    width=900,
    height=700,
    font=dict(size=12)
)

fig2.show()

print("-" * 40)
print(" 可视化完成")

 正在生成文本段落风格空间...
----------------------------------------


----------------------------------------
 可视化完成


---
<a id='part7'></a>
# 第七部分: 作家风格热力图

展示作家之间的风格相似度矩阵。

In [23]:
#@title ## 7.1 作家风格热力图 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，生成作家风格相似度热力图。
#@markdown
#@markdown ### 解读指南
#@markdown - 颜色越红，风格越相似
#@markdown - 颜色越蓝，风格差异越大
#@markdown - 对角线是自己和自己的相似度（100%）
#@markdown
#@markdown ### 观察要点
#@markdown - 哪两位作家颜色最红？说明风格最接近
#@markdown - 哪两位作家颜色最蓝？说明风格差异最大
#@markdown - 古典文献和民国作家之间的颜色如何？
#@markdown ---

import plotly.graph_objects as go

# 获取作家列表
authors = list(author_embeddings.keys())

# 构建相似度矩阵
sim_matrix = []
for a1 in authors:
    row = []
    for a2 in authors:
        sim = np.dot(author_embeddings[a1], author_embeddings[a2]) / (
            np.linalg.norm(author_embeddings[a1]) * np.linalg.norm(author_embeddings[a2])
        )
        row.append(sim)
    sim_matrix.append(row)

# 绘制热力图
fig = go.Figure(data=go.Heatmap(
    z=sim_matrix,
    x=authors,
    y=authors,
    colorscale='RdBu_r',
    zmid=0.5,
    hovertemplate='%{y} vs %{x}<br>相似度: %{z:.1%}<extra></extra>'
))

fig.update_layout(
    title='作家风格相似度热力图',
    width=900,
    height=800,
    xaxis={'tickangle': 45}
)

fig.show()

# 输出最相似和最不相似的作家组合
print("\n" + "=" * 50)
print(" 风格分析统计")
print("=" * 50)

# 找出最相似的组合（排除自己和自己）
max_sim = 0
max_pair = ("", "")
min_sim = 1
min_pair = ("", "")

for i, a1 in enumerate(authors):
    for j, a2 in enumerate(authors):
        if i < j:  # 避免重复和对角线
            sim = sim_matrix[i][j]
            if sim > max_sim:
                max_sim = sim
                max_pair = (a1, a2)
            if sim < min_sim:
                min_sim = sim
                min_pair = (a1, a2)

print(f"\n 最相似: {max_pair[0]} & {max_pair[1]} ({max_sim:.1%})")
print(f" 最不相似: {min_pair[0]} & {min_pair[1]} ({min_sim:.1%})")
print("=" * 50)


 风格分析统计

 最相似: 史铁生 & 朱自清 (97.6%)
 最不相似: 冯骥才 & 论语 (69.3%)


---
<a id='part8'></a>
# 第八部分: AI风格解说员

使用大语言模型解读作家风格差异的文化原因。

In [24]:
#@title ## 8.1 加载Yi大语言模型 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，加载Yi大语言模型。
#@markdown
#@markdown ### 关于Yi-1.5-6B
#@markdown **Yi** 是零一万物开发的开源中文大语言模型：
#@markdown - **开发者**：零一万物（李开复创办）
#@markdown - **参数量**：60亿（6B）
#@markdown - **特点**：中文能力优秀，兼容性好
#@markdown - **显存需求**：约12GB
#@markdown ---

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

LLM_LOADED = False
llm_model = None
llm_tokenizer = None

print(" 正在加载Yi-1.5-6B模型...")
print(" (首次运行需下载约12GB，请耐心等待)")
print("-" * 40)

try:
    model_name = "01-ai/Yi-1.5-6B-Chat"

    llm_tokenizer = AutoTokenizer.from_pretrained(model_name)
    llm_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )

    LLM_LOADED = True
    print(" 模型加载成功！")

except Exception as e:
    print(f" 模型加载失败: {e}")
    print(" 将使用预设回复模式")

print("-" * 40)

 正在加载Yi-1.5-6B模型...
 (首次运行需下载约12GB，请耐心等待)
----------------------------------------


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/2.21G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

 模型加载成功！
----------------------------------------


In [25]:
#@title ## 8.2 AI风格解说员 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，启动AI风格解说功能。
#@markdown
#@markdown ### 使用方法
#@markdown 选择两位作家，AI会生成对比分析报告，解释风格差异的文化原因。
#@markdown
#@markdown ### 观察要点
#@markdown - AI分析的维度是否合理？
#@markdown - AI的解读与你的理解有何异同？
#@markdown - AI能否捕捉到时代背景的影响？
#@markdown ---

def explain_style_difference(author1, author2):
    """解释两位作家的风格差异"""

    # 获取相似度
    if author1 in df_similarity.index and author2 in df_similarity.columns:
        sim = df_similarity.loc[author1, author2]
    else:
        sim = 0.5

    # 获取样本文本
    sample1 = df_sampled[df_sampled['author'] == author1].iloc[0]['content'][:400] if author1 in df_sampled['author'].values else ""
    sample2 = df_sampled[df_sampled['author'] == author2].iloc[0]['content'][:400] if author2 in df_sampled['author'].values else ""

    prompt = f"""你是一位资深的中国现当代文学研究专家。请对比分析{author1}和{author2}的写作风格。

【数据参考】
AI计算的风格相似度：{sim:.1%}

【{author1}作品片段】
{sample1[:300]}

【{author2}作品片段】
{sample2[:300]}

【分析要求】
请严格基于上述文本片段，从以下四个维度进行对比分析：

1. 句式结构
   - 句子长短倾向（短句为主/长句为主/长短交错）
   - 句式类型（陈述句/疑问句/感叹句的使用频率）
   - 请引用原文中的具体句子作为例证

2. 词汇特征
   - 用词风格（口语化/书面化/文言化）
   - 特色词汇（请列举片段中有代表性的词语）
   - 修辞手法（如有，请指出具体位置）

3. 情感基调
   - 整体氛围（冷静客观/热烈抒情/忧郁沉思/幽默讽刺）
   - 作者与读者的距离感（亲近/疏离）
   - 情感表达方式（直抒胸臆/含蓄内敛）

4. 文化印记
   - 时代特征（能否看出写作年代的痕迹）
   - 地域特色（如有）
   - 中西文化元素（古典传统/现代白话/外来影响）

【注意事项】
- 只分析给定的文本片段，不要引入片段以外的作品内容
- 每个维度用2-3句话概括，避免冗长
- 如果某个维度在片段中体现不明显，请如实说明
- 输出总结后立即停止，不要继续生成任何内容

【总结要求】
最后请用一句话概括：{author1}的风格是____，{author2}的风格是____，最大差异在于____。"""

    if LLM_LOADED:
        try:
            messages = [{"role": "user", "content": prompt}]
            input_ids = llm_tokenizer.apply_chat_template(messages, return_tensors="pt").to(llm_model.device)
            outputs = llm_model.generate(
                input_ids,
                max_new_tokens=800,
                do_sample=True,
                temperature=0.7,
                eos_token_id=llm_tokenizer.eos_token_id,
                pad_token_id=llm_tokenizer.pad_token_id
            )
            full_response = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)

            # 提取assistant回复
            if "assistant" in full_response:
                response = full_response.split("assistant")[-1].strip()
            else:
                response = full_response

            # 截断重复内容
            if "最大差异在于" in response:
                # 找到总结句的结尾
                idx = response.find("最大差异在于")
                end_idx = response.find("。", idx)
                if end_idx != -1:
                    response = response[:end_idx+1]

            # 去除可能的重复段落
            if "本文档" in response:
                response = response.split("本文档")[0].strip()

            return response
        except Exception as e:
            return f"生成失败: {e}"
    else:
        return f"""[AI风格解说 - {author1} vs {author2}]

风格相似度：{sim:.1%}

1. 句式结构：
   - 需要加载大语言模型才能生成详细分析

2. 词汇选择：
   - 请运行8.1单元格加载Qwen模型

3. 文化底色：
   - 模型加载后可获得深度分析

4. 情感表达：
   - 等待模型加载...

提示：请先运行8.1单元格加载Qwen模型"""

# 创建Gradio界面
try:
    author_list = list(author_embeddings.keys())
except:
    author_list = ['鲁迅', '老舍', '巴金']

demo_explainer = gr.Interface(
    fn=explain_style_difference,
    inputs=[
        gr.Dropdown(choices=author_list, value=author_list[0], label='作家1'),
        gr.Dropdown(choices=author_list, value=author_list[1] if len(author_list) > 1 else author_list[0], label='作家2')
    ],
    outputs=gr.Textbox(label='AI分析报告', lines=20),
    title='AI风格解说员',
    description='选择两位作家，AI会分析他们的风格差异并解释可能的文化原因。',
    allow_flagging='never'
)

demo_explainer.launch(share=True, debug=False, quiet=True)

* Running on public URL: https://d6c8f3661b6efc087d.gradio.live


---
<a id='part9'></a>
# 第九部分: AI模仿作家

让AI模仿特定作家的风格写作。

In [26]:
#@title ## 9.1 AI模仿作家 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，让AI模仿特定作家的风格写作。
#@markdown
#@markdown ### 使用方法
#@markdown 1. 选择要模仿的作家
#@markdown 2. 输入写作主题
#@markdown 3. AI会生成该作家风格的文字
#@markdown
#@markdown ### 思考题
#@markdown - AI模仿的像不像？哪些方面像，哪些方面不像？
#@markdown - AI能学会作家的语言风格，但能学会其思想深度吗？
#@markdown - 这种技术可能带来哪些正面和负面影响？
#@markdown ---

def imitate_author(author, topic):
    """模仿作家风格写作"""

    if not topic.strip():
        return "请输入写作主题"

    # 获取作家样本
    author_samples = df_sampled[df_sampled['author'] == author]
    if len(author_samples) == 0:
        sample_text = ""
    else:
        sample_text = author_samples.iloc[0]['content'][:300]

    # 根据作家设定不同的语气
    tone_map = {
        # 民国作家
        '鲁迅': '冷峻讽刺、犀利批判',
        '老舍': '幽默市井、京味浓郁',
        '巴金': '热烈激昂、真挚深情',
        '三毛': '温柔流浪、浪漫洒脱',
        '冰心': '清新温婉、母爱童真',
        '朱自清': '细腻抒情、清丽雅致',
        '萧红': '质朴忧伤、苍凉悲悯',
        '郁达夫': '感伤颓废、坦诚自剖',
        '周作人': '闲适冲淡、知识趣味',
        '林徽因': '灵动诗意、典雅浪漫',
        '杨绛': '淡泊从容、幽默睿智',
        '许地山': '朴素哲理、宗教意味',
        '胡适': '平易理性、清晰明快',
        '季羡林': '朴实真诚、学者情怀',
        '冯骥才': '细腻温情、民俗关怀',
        '史铁生': '沉思内省、生命追问',
        '章诒和': '典雅怀旧、世家风骨',
        # 古典文献
        '论语': '简洁庄重、哲理深邃',
        '诗经': '质朴天真、反复咏叹',
    }
    tone = tone_map.get(author, '沉静克制')

    prompt = f"""任务：写一段全新原创的中文散文片段。
主题：{topic}
风格：模仿{author}的写作气质

【写作约束】
1. 视角：第一人称"我"，语气{tone}
2. 句式：长短句交错，可用破折号、省略号制造停顿
3. 内容：加入具体的生活细节和感官描写
4. 禁止：不提作家名字、不引用名句、不说"总之""因此"、不写心灵鸡汤式的感悟

【输出要求】
- 严格控制在150-200字
- 写完即停，不要超出字数
- 只输出正文，不要任何说明

直接开始："""

    if LLM_LOADED:
        try:
            messages = [{"role": "user", "content": prompt}]
            input_ids = llm_tokenizer.apply_chat_template(messages, return_tensors="pt").to(llm_model.device)
            outputs = llm_model.generate(input_ids, max_new_tokens=400, do_sample=True, temperature=0.8)
            full_response = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)

            # 提取assistant回复
            if "assistant" in full_response:
                response = full_response.split("assistant")[-1].strip()
            else:
                response = full_response

            # 确保在完整句子处结束
            for end_char in ['。', '！', '？', '"', '…']:
                last_idx = response.rfind(end_char)
                if last_idx != -1 and last_idx > len(response) * 0.7:  # 至少保留70%内容
                    response = response[:last_idx+1]
                    break

            result = []
            result.append("=" * 50)
            result.append(f" AI模仿{author}风格")
            result.append(f" 主题：{topic}")
            result.append("=" * 50)
            result.append("")
            result.append(response)
            result.append("")
            result.append("=" * 50)

            return "\n".join(result)
        except Exception as e:
            return f"生成失败: {e}"
    else:
        return f"""[需要加载大语言模型]

要使用AI模仿功能，请先运行8.1单元格加载Yi模型。

模型加载后，AI将能够：
- 学习{author}的语言风格
- 模仿其句式结构和用词习惯
- 生成关于「{topic}」的仿写文字"""

# 创建Gradio界面
try:
    author_list = list(author_embeddings.keys())
except:
    author_list = ['鲁迅', '老舍', '巴金']

demo_imitation = gr.Interface(
    fn=imitate_author,
    inputs=[
        gr.Dropdown(choices=author_list, value='鲁迅', label='选择作家'),
        gr.Textbox(label='写作主题', placeholder='例如：手机、网络、城市生活...')
    ],
    outputs=gr.Textbox(label='AI生成文字', lines=15),
    title='AI模仿作家',
    description='选择一位作家，输入主题，AI会模仿该作家的风格进行写作。',
    examples=[
        ['鲁迅', '手机'],
        ['鲁迅', '网络喷子'],
        ['鲁迅', '加班文化'],
    ],
    allow_flagging='never'
)

demo_imitation.launch(share=True, debug=False, quiet=True)

* Running on public URL: https://3471a237f826ff8078.gradio.live


---
<a id='appendix'></a>
# 附录: 常见问题与拓展

## 常见问题

**Q1: 为什么某些作家的识别准确率较低?**

可能原因:
- 该作家的作品数量较少，样本不足
- 该作家的风格与其他作家相似
- 该作家在不同时期、不同体裁的作品风格差异大

**Q2: AI计算的「风格相似度」真的准确吗?**

AI计算的相似度基于文本的语义向量，主要捕捉:
- 词汇使用习惯
- 句式结构特点
- 主题和意象分布

但无法完全捕捉:
- 思想深度
- 历史语境
- 读者的主观感受

**Q3: 为什么AI模仿的文字有时不太像?**

小参数模型（6B）的局限:
- 对作家风格的理解较浅
- 容易照搬参考文本的人名地名
- 难以把握深层的思想内涵

可尝试的改进:
- 提供更多、更典型的参考文本
- 调整生成参数（temperature等）
- 使用更大的模型（如70B）

**Q4: 如何添加更多作家的作品?**

修改第一部分的数据加载代码，添加新的数据源即可。

## 拓展方向

1. **情感分析**: 分析作家作品中的情感变化轨迹
2. **主题建模**: 使用LDA等方法提取作家的核心主题
3. **时序分析**: 分析作家风格随时间的演变
4. **影响关系**: 构建作家之间的影响网络
5. **跨语言对比**: 对比翻译作品与原作的风格差异

## 参考资料

- BGE中文嵌入模型: https://huggingface.co/BAAI/bge-large-zh-v1.5
- Yi大语言模型: https://huggingface.co/01-ai/Yi-1.5-6B-Chat
- 鲁迅全集: https://github.com/Ac-heron/luxun
- 中国古诗词数据库: https://github.com/chinese-poetry/chinese-poetry

---

*本教学Notebook基于开源数据和模型开发，仅供教学与研究使用。*

*如有问题或建议，欢迎反馈！*